In this notebook we have presented a demo code snippets using the Telescope Dataset which has 2 classes as its labels and hence a binary classification task. We have demonstrated with ANN for the purpose of this notebook, which can be modified as required.

We have presented the demo code for each of the following techniques:

1.   Post Training Pruning
2.   Post Training Quantization
3.   Multi-Point Quantization
4.   Quantization Aware Training





## 1: Setting up the Colab Notebook and Google Drive paths

In [1]:
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)
MY_GOOGLE_DRIVE_PATH = 'My Drive/Capstone_Prasham/'

from os.path import join
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)

%cd "{PROJECT_PATH}"
%cd "Edge"

/content/drive
Mounted at /content/drive
/content/drive/My Drive/Capstone_Prasham
/content/drive/My Drive/Capstone_Prasham/Edge


## 2: Importing required library functions

In [2]:
import os
import time
from collections import Counter
from datetime import datetime

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from torch.utils.data import Dataset, Subset
from tqdm import trange
from tqdm.auto import tqdm, trange

from utils.util_functions import *

#Setting the Random Seed to ensure reproducibility of the results
np.random.seed(0)
torch.manual_seed(0)

##3: Defining the Dataset

We need to define the dataset class which would be used to extract data and obtain the train and test datasets.

In [3]:
class TelescopeDataset(Dataset):
    """ Pytorch dataset for Telescope """

    def __init__(self, csv_file = 'telescope.dat', path = 'data/'):
        """
        constructor to load a csv, preprocess it into torch Dataset
        """

        self.dataset = pd.read_table(path + csv_file, header=None, delimiter=',')
        self.dataset.columns = ['FLength', 'FWidth', 'FSize', 'FConc', 'FConc1', 'FAsym',
                     'FM3Long', 'FM3Trans', 'FAlpha', 'FDist', 'Class']

        scaler = StandardScaler()
        data = scaler.fit_transform(self.dataset.iloc[:, :-1])
        target = LabelEncoder().fit_transform(self.dataset.Class)
        self.x = data.astype(np.float32)
        self.y = target.astype(np.long)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):

      sample_x = self.x[idx]
      sample_y = np.array([self.y[idx]])
      return (torch.from_numpy(sample_x), torch.from_numpy(sample_y))

## 4: Defining the model, training and evaluation methods

In here we define the model class which returns a Dense Neural Network with specified values for setting the architecture of the network.

In [4]:
class DenseNeuralNet(nn.Module):
  """
  Class to create the architecture of dense neural network and the forward pass of batch
  """
  def __init__(self, input_size, num_classes,layers=[],dropout_prob = 0.0, batch_norm = False):
    super().__init__()
    layers.append(num_classes)
    config = {
        "num_layers" : len(layers),
        "nodes_per_layer" : layers,
        "dropout_prob" : dropout_prob,
        "batch_norm" : batch_norm
    }

    layers = []
    prev = input_size

    for i in range(0, config["num_layers"]):
      layers.append(nn.Linear(prev, config["nodes_per_layer"][i]))
      if not i==config["num_layers"]-1:
        if config["batch_norm"]:
          layers.append(nn.BatchNorm1d(num_features = config["nodes_per_layer"][i]))
        layers.append(nn.ReLU())

        if config["dropout_prob"] > 0:
          layers.append(nn.Dropout(p = config["dropout_prob"]))

      prev = config["nodes_per_layer"][i]

    self.net = nn.Sequential(*layers)


  def forward(self, x):
      out = self.net(x)
      return out

We further define a method that takes the model as its parameter along with the specified hyperparameters for training and runs the forward pass and backpropagation to train the model.

In [5]:
def train(model, train_set, val_set, test_set , batch_size = 16, learning_rate = 0.03, epochs = 5, eval_steps = 10, skip_train_set = True):
  criterion = nn.CrossEntropyLoss()

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  model = model.to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  time_stamp = str(datetime.now())
  filename = "Telescope_Dataset_" + time_stamp
  
  train_loader = torch.utils.data.DataLoader(dataset= train_set, batch_size=batch_size, shuffle=True)
  global_step = 0
  for ep in tqdm(range(epochs), desc = ' Epoch Progress:', ncols=900):
    train_iterator = tqdm(train_loader, desc = 'Train Iteration for epoch:'+ str(ep+1), ncols=900)    
    for step, inputs in enumerate(train_iterator):
      model.train()
      model.zero_grad()

      global_step +=1
      x = inputs[0]
      y = torch.squeeze(inputs[1],1).long()
      x = x.to(device)
      y = y.to(device)

      logits = model(x)
      loss = criterion(logits, y)
      loss.backward()
      optimizer.step()

      
    val_loss, val_accuracy = evaluate(model, val_set, batch_size, criterion, ep)
    
    if not skip_train_set:
      train_loss , train_accuracy = evaluate(model, train_set, batch_size, criterion, ep)
      print("Step = %d, training loss =  %f, training accuracy = %f" %(global_step, train_loss, train_accuracy))
    print("Step = %d, validation loss =  %f, validation accuracy = %f" %(global_step, val_loss, val_accuracy))
    
  test_loss, test_accuracy = evaluate(model, test_set, batch_size, criterion, ep)
  print("End of Training, test loss =  %f, test accuracy = %f" %(test_loss, test_accuracy))

We also define a method to evaluate the model on the specified test set. The function returns the loss and accuracy for the given model on the given test set.

In [6]:
def evaluate(model, test_set, batch_size, criterion, ep):
  test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size = batch_size, shuffle=True)
  test_iterator = tqdm(test_loader, desc = 'Eval Iteration for epoch:'+str(ep+1), ncols = 900)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  
  model.eval()
  global_step = 0
  total_correct = 0
  total_samples = 0
  total_loss = 0.0
  for step, inputs in enumerate(test_iterator):
      global_step +=1
      x = inputs[0]
      y = torch.squeeze(inputs[1],1).long()
      x = x.to(device)
      y = y.to(device)

      logits = model(x)
      loss = criterion(logits, y)
      correct, samples = get_accuracy(logits, y)
      total_correct +=correct.item()
      total_samples +=samples
      total_loss +=loss

  acc = total_correct / total_samples
  total_loss = total_loss / global_step
  
  return (total_loss, acc)

## 5: Training the Model

For purpose of running post-training methods we need to train the model first and then apply the required method. We use the appropriate criterion and optimizer for this purpose. We will further store this model so that we can load the model and reuse it for demonstrating different techniques.

In [7]:
# setting the hyperparameters
input_dim =  10
output_classes = 2
learning_rate = 0.001
batch_size = 16
epochs = 10
eval_steps = 100

# Defining the model name and directory
model_name = 'model.pt'
model_dir = 'model_artifacts'

#Loading the dataset and splitting it into train, validation and test set
telescope_dataset = TelescopeDataset()
train_set, val_set, test_set = get_get_train_val_test(telescope_dataset, val_split=0.40)

In [8]:
# Training the Model
print("Training Model")
model = DenseNeuralNet(input_size = input_dim, 
                              num_classes = output_classes,
                              layers = [10],
                              dropout_prob=0,
                              batch_norm=False)   
print("-------------------------------------------------------")
print(model)
print("-------------------------------------------------------")
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train(model = model,
      train_set = train_set, 
      val_set = val_set, 
      test_set = test_set , 
      batch_size = batch_size, 
      learning_rate = learning_rate, 
      epochs = epochs, 
      eval_steps = eval_steps,
      skip_train_set=False)

# Saving the model
torch.save(model, os.path.join(model_dir, model_name))

Training Model
-------------------------------------------------------
DenseNeuralNet(
  (net): Sequential(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=2, bias=True)
  )
)
-------------------------------------------------------



Step = 714, training loss =  0.412531, training accuracy = 0.807483
Step = 714, validation loss =  0.425745, validation accuracy = 0.800473



Step = 1428, training loss =  0.397205, training accuracy = 0.819751
Step = 1428, validation loss =  0.409000, validation accuracy = 0.809674



Step = 2142, training loss =  0.385362, training accuracy = 0.825447
Step = 2142, validation loss =  0.398041, validation accuracy = 0.814932



Step = 2856, training loss =  0.373484, training accuracy = 0.835436
Step = 2856, validation loss =  0.385252, validation accuracy = 0.827024


...


Step = 4998, training loss =  0.351196, training accuracy = 0.855853
Step = 4998, validation loss =  0.360388, validation accuracy = 0.849632



Step = 5712, training loss =  0.345659, training accuracy = 0.859534
Step = 5712, validation loss =  0.355944, validation accuracy = 0.853838



Step = 6426, training loss =  0.341287, training accuracy = 0.862163
Step = 6426, validation loss =  0.351795, validation accuracy = 0.856467



Step = 7140, training loss =  0.338536, training accuracy = 0.863214
Step = 7140, validation loss =  0.348268, validation accuracy = 0.859359




End of Training, test loss =  0.344486, test accuracy = 0.858833


## 6: Post-Training Pruning

This section demonstartes the use of methods to prune the network. We make the required import and demonstrate the use of the methods on the ANN model we trained in the previous section.

In [9]:
from utils.post_training_pruning.pruning import prune_model, pruning_multiple

In [10]:
pruned_model = prune_model(model,prune_percentage=0.40)

In [11]:
loss, accuracy = evaluate(model = model,test_set = test_set,batch_size=16,criterion=criterion,ep=0)
print("Accuracy for normal model:",accuracy)
loss, accuracy = evaluate(model = pruned_model,test_set = test_set,batch_size=16,criterion=criterion,ep=0)
print("Accuracy for pruned model:",accuracy)


Accuracy for normal model: 0.8588328075709779



Accuracy for pruned model: 0.8393796004206099


Following method returns a Dataframe containing the pruned model artifact along with the specifies levels of pruning. It also has columns for train, test loss and accuracy with NaNs as values across all rows. We will fill those entries using the evaluate function defined in section 4.

In [12]:
pruned_model_multiple_levels = pruning_multiple(model_name=model_name, prune_percentage=[0.30,0.40,0.50,0.60])

In [13]:
pruned_model_multiple_levels

,model,pruning_percentage,model_artifact,pruned_model_artifact,train_loss,train_acc,test_loss,test_acc
0,model.pt,0.3,DenseNeuralNet(\n (net): Sequential(\n (0)...,DenseNeuralNet(\n (net): Sequential(\n (0)...,NaN,NaN,NaN,NaN
1,model.pt,0.4,DenseNeuralNet(\n (net): Sequential(\n (0)...,DenseNeuralNet(\n (net): Sequential(\n (0)...,NaN,NaN,NaN,NaN
2,model.pt,0.5,DenseNeuralNet(\n (net): Sequential(\n (0)...,DenseNeuralNet(\n (net): Sequential(\n (0)...,NaN,NaN,NaN,NaN
3,model.pt,0.6,DenseNeuralNet(\n (net): Sequential(\n (0)...,DenseNeuralNet(\n (net): Sequential(\n (0)...,NaN,NaN,NaN,NaN


In [14]:
# Filling up the columns for train, test acuracy and loss using the evaluate method.
train_loss_list = []
train_accuracy_list = []
test_loss_list = []
test_accuracy_list = []
for i in pruned_model_multiple_levels["pruned_model_artifact"]:
  train_loss, train_accuracy = evaluate(model=i, 
                                      test_set = train_set,
                                      batch_size=batch_size, 
                                      criterion=criterion,
                                      ep=0) 
  test_loss, test_accuracy = evaluate(model=i, 
                                      test_set = test_set,
                                      batch_size=batch_size, 
                                      criterion=criterion,
                                      ep=0)  
  train_loss_list.append(train_loss.item())
  test_loss_list.append(test_loss.item())
  train_accuracy_list.append(train_accuracy)
  test_accuracy_list.append(test_accuracy)
pruned_model_multiple_levels["train_loss"] = train_loss_list
pruned_model_multiple_levels["train_acc"] = train_accuracy_list
pruned_model_multiple_levels["test_loss"] = test_loss_list
pruned_model_multiple_levels["test_acc"] = test_accuracy_list

In [15]:
pruned_model_multiple_levels

,model,pruning_percentage,model_artifact,pruned_model_artifact,train_loss,train_acc,test_loss,test_acc
0,model.pt,0.3,DenseNeuralNet(\n (net): Sequential(\n (0)...,DenseNeuralNet(\n (net): Sequential(\n (0)...,0.353822,0.857606,0.358391,0.854890
1,model.pt,0.4,DenseNeuralNet(\n (net): Sequential(\n (0)...,DenseNeuralNet(\n (net): Sequential(\n (0)...,0.371074,0.850245,0.378859,0.839380
2,model.pt,0.5,DenseNeuralNet(\n (net): Sequential(\n (0)...,DenseNeuralNet(\n (net): Sequential(\n (0)...,0.393714,0.845952,0.401741,0.835436
3,model.pt,0.6,DenseNeuralNet(\n (net): Sequential(\n (0)...,DenseNeuralNet(\n (net): Sequential(\n (0)...,0.487870,0.772082,0.497234,0.755783


##7: Post-Training Quantization 

This section demonstartes the use of methods to quantize the network. We make the required import and demonstrate the use of the methods on the ANN model we trained in section 5.

In [16]:
from utils.post_training_quantization.rounding import quantization

In [17]:
quantized_models = quantization(model_name=model_name,precision=[4],unique_val_method=["uniform_range"])
quantized_model_artifact = quantized_models["model_artifact"][0]
quantized_model_artifact

DenseNeuralNet(
  (net): Sequential(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=2, bias=True)
  )
)

In [18]:
loss, accuracy = evaluate(model = model,test_set = test_set,batch_size=16,criterion=criterion,ep=0)
print("Accuracy for normal model:",accuracy)
loss, accuracy = evaluate(model = quantized_model_artifact,test_set = test_set,batch_size=16,criterion=criterion,ep=0)
print("Accuracy for Quantized model:",accuracy)


Accuracy for normal model: 0.8588328075709779



Accuracy for Quantized model: 0.8554153522607781


Following method returns a Dataframe containing the quantized model artifact along with the specified levels of precision (Default is [16, 12, 8, 6, 4]). It runs the quantization for all the methods for generating unique values. It also has columns for train, test loss and accuracy with NaNs as values across all rows. We will fill those entries using the evaluate function defined in section 4.

In [19]:
quantized_models_results = quantization(model_name=model_name)

In [20]:
# Filling up the columns for train, test acuracy and loss using the evaluate method.
train_loss_list = []
train_accuracy_list = []
test_loss_list = []
test_accuracy_list = []
for i in quantized_models_results["model_artifact"]:
  train_loss, train_accuracy = evaluate(model=i, 
                                      test_set = train_set,
                                      batch_size=batch_size, 
                                      criterion=criterion,
                                      ep=0) 
  test_loss, test_accuracy = evaluate(model=i, 
                                      test_set = test_set,
                                      batch_size=batch_size, 
                                      criterion=criterion,
                                      ep=0)  
  train_loss_list.append(train_loss.item())
  test_loss_list.append(test_loss.item())
  train_accuracy_list.append(train_accuracy)
  test_accuracy_list.append(test_accuracy)
quantized_models_results["train_loss"] = train_loss_list
quantized_models_results["train_acc"] = train_accuracy_list
quantized_models_results["test_loss"] = test_loss_list
quantized_models_results["test_acc"] = test_accuracy_list

In [21]:
quantized_models_results

,model,quant_method,bin_method,precision,model_artifact,train_loss,train_acc,test_loss,test_acc
0,model.pt,normal_rounding,uniform_range,16,DenseNeuralNet(\n (net): Sequential(\n (0)...,0.338442,0.863214,0.344914,0.858833
1,model.pt,normal_rounding,uniform_range,12,DenseNeuralNet(\n (net): Sequential(\n (0)...,0.338512,0.863127,0.344416,0.858833
2,model.pt,normal_rounding,uniform_range,10,DenseNeuralNet(\n (net): Sequential(\n (0)...,0.338287,0.863127,0.344532,0.858833
3,model.pt,normal_rounding,uniform_range,8,DenseNeuralNet(\n (net): Sequential(\n (0)...,0.338388,0.864266,0.344754,0.858570
4,model.pt,normal_rounding,uniform_range,6,DenseNeuralNet(\n (net): Sequential(\n (0)...,0.338535,0.864353,0.344425,0.858044
5,model.pt,normal_rounding,uniform_range,4,DenseNeuralNet(\n (net): Sequential(\n (0)...,0.344925,0.859709,0.350436,0.855415
6,model.pt,normal_rounding,prior_normal,16,DenseNeuralNet(\n (net): Sequential(\n (0)...,0.338741,0.863214,0.344448,0.858833
7,model.pt,normal_rounding,prior_normal,12,DenseNeuralNet(\n (net): Sequential(\n (0)...,0.338240,0.862951,0.344478,0.858833
8,model.pt,normal_rounding,prior_normal,10,DenseNeuralNet(\n (net): Sequential(\n (0)...,0.340849,0.860936,0.345569,0.855678
9,model.pt,normal_rounding,prior_normal,8,DenseNeuralNet(\n (net): Sequential(\n (0)...,0.347385,0.856730,0.350518,0.853049


## 8: Multi-Point Quantization

This section demonstartes the use of methods to quantize (multi-point post training quantization) the network. We make the required import and demonstrate the use of the methods on the ANN model we trained in section 5.

In [22]:
from utils.post_training_quantization.multipoint_quantization import multipoint_quantization

In [23]:
multipoint_quantized = multipoint_quantization(model_name = model_name, precision = [4])
multipoint_quantized_model = multipoint_quantized[multipoint_quantized["precision"]==4]["model_artifact"][1]
multipoint_quantized_model


--------Quantizing the model model.pt with precision 4
All layers except bias layers:  ['net.0.weight', 'net.2.weight']

Quantizing layer:net.2.weight , with weights shape:torch.Size([2, 10])
Final error of W quantization: 2.210700273513794
--------Results appended for model.pt with precision 4



DenseNeuralNet(
  (net): Sequential(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=2, bias=True)
  )
)

In [24]:
loss, accuracy = evaluate(model = model,test_set = test_set,batch_size=16,criterion=criterion,ep=0)
print("Accuracy for normal model:",accuracy)
loss, accuracy = evaluate(model = quantized_model_artifact,test_set = test_set,batch_size=16,criterion=criterion,ep=0)
print("Accuracy for Quantized model:",accuracy)


Accuracy for normal model: 0.8588328075709779



Accuracy for Quantized model: 0.8554153522607781


Following method returns a Dataframe containing the quantized model artifact along with the specified levels of precision. It runs the quantization for all the methods for generating unique values. It also has columns for train, test loss and accuracy with NaNs as values across all rows. We will fill those entries using the evaluate function defined in section 4.

In [25]:
multi_point_quantized_models_results = multipoint_quantization(model_name = model_name,precision = [8,6,4,2])


--------Quantizing the model model.pt with precision 8
All layers except bias layers:  ['net.0.weight', 'net.2.weight']

Quantizing layer:net.2.weight , with weights shape:torch.Size([2, 10])
Final error of W quantization: 0.0008451592875644565
--------Results appended for model.pt with precision 8


--------Quantizing the model model.pt with precision 6
All layers except bias layers:  ['net.0.weight', 'net.2.weight']

Quantizing layer:net.2.weight , with weights shape:torch.Size([2, 10])
Final error of W quantization: 0.7478877902030945
--------Results appended for model.pt with precision 6


--------Quantizing the model model.pt with precision 4
All layers except bias layers:  ['net.0.weight', 'net.2.weight']

Quantizing layer:net.2.weight , with weights shape:torch.Size([2, 10])
Final error of W quantization: 2.210700273513794
--------Results appended for model.pt with precision 4


--------Quantizing the model model.pt with precision 2
All layers except bias layers:  ['net.0.weigh

In [26]:
# Filling up the columns for train, test acuracy and loss using the evaluate method.
train_loss_list = []
train_accuracy_list = []
test_loss_list = []
test_accuracy_list = []
for i in multi_point_quantized_models_results["model_artifact"]:
  train_loss, train_accuracy = evaluate(model=i, 
                                      test_set = train_set,
                                      batch_size=batch_size, 
                                      criterion=criterion,
                                      ep=0) 
  test_loss, test_accuracy = evaluate(model=i, 
                                      test_set = test_set,
                                      batch_size=batch_size, 
                                      criterion=criterion,
                                      ep=0)  
  train_loss_list.append(train_loss.item())
  test_loss_list.append(test_loss.item())
  train_accuracy_list.append(train_accuracy)
  test_accuracy_list.append(test_accuracy)
multi_point_quantized_models_results["train_loss"] = train_loss_list
multi_point_quantized_models_results["train_acc"] = train_accuracy_list
multi_point_quantized_models_results["test_loss"] = test_loss_list
multi_point_quantized_models_results["test_acc"] = test_accuracy_list

In [27]:
multi_point_quantized_models_results

,model,quant_method,precision,model_artifact,train_loss,train_acc,test_loss,test_acc
0,model.pt,multi-point,32,DenseNeuralNet(\n (net): Sequential(\n (0)...,0.339505,0.863214,0.344224,0.858833
1,model.pt,multi-point,8,DenseNeuralNet(\n (net): Sequential(\n (0)...,0.338059,0.863214,0.344599,0.858833
2,model.pt,multi-point,6,DenseNeuralNet(\n (net): Sequential(\n (0)...,0.373080,0.853488,0.373159,0.847529
3,model.pt,multi-point,4,DenseNeuralNet(\n (net): Sequential(\n (0)...,0.675195,0.399667,0.665774,0.416930
4,model.pt,multi-point,2,DenseNeuralNet(\n (net): Sequential(\n (0)...,0.840309,0.346302,0.830236,0.360147


## 9: Quantization Aware Training

This section demonstartes the development of the methods to achieve Quantization Aware Training. We need to update the the train and evaluate methods to accumulate the QAT settings and have demonstrated the required changes in this section.

In [28]:
from utils.quantization_aware_training.qat import *

Training Loop with Quantization Aware Training after every minibatch

In [29]:
def qat_train(model, train_set, val_set, test_set , batch_size = 16, learning_rate = 0.03, epochs = 5, eval_steps = 10, skip_train_set = True, qat_mode = False, qat_params = None):
  # GPU/CPU use
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  # print("Device: ", device)
  model = model.to(device)

  # define loss & optimizer
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  # iterate over epoch
  weights_fp = clone_model_weights(model)
  train_loader = torch.utils.data.DataLoader(dataset= train_set, batch_size=batch_size, shuffle=True, num_workers=1)
  global_step = 0
  for ep in tqdm(range(epochs), desc = ' Epoch Progress:', ncols=900):
    train_iterator = tqdm(train_loader, desc = 'Train Iteration for epoch:'+ str(ep+1), ncols=900)    
    running_loss = 0

    # iterate over batches
    for step, inputs in enumerate(train_iterator):
      model.train()
      global_step +=1
      optimizer.zero_grad()
      if qat_mode:
          w_fp_min_max, w_q_min_max = quantize_model(model, weights_fp, qat_params)
      
      x, y = inputs[0].to(device), torch.squeeze(inputs[1],1).long().to(device)
      logits = model(x)
      loss = criterion(logits, y)
      loss.backward()

      if qat_mode:
        update_gradients(model, weights_fp, w_fp_min_max, w_q_min_max)

      optimizer.step()

      if qat_mode:  
        weights_fp = clone_model_weights(model)
  
      running_loss+=loss.item()

    # find validation accuracy
    val_loss, val_accuracy = qat_evaluate(model, val_set, batch_size, criterion, ep)
    print("Step = %d, validation loss =  %.3f, validation accuracy = %.3f" %(global_step, val_loss, val_accuracy))
    
    # find train accuracy if needed
    if not skip_train_set:
      train_loss , train_accuracy = qat_evaluate(model, train_set, batch_size, criterion, ep)
      print("Step = %d, training loss =  %.3f, training accuracy = %.3f" %(global_step, train_loss, train_accuracy))

  # find test accuracy with final model
  if test_set is not None:  
    test_loss, test_accuracy = qat_evaluate(model, test_set, batch_size, criterion, ep)
    print("End of Training, test loss =  %.3f, test accuracy = %.3f" %(test_loss, test_accuracy))

Evaluation loop for QAT (Note : model is stored in full precision and quantized before evaluation)

In [30]:
def qat_evaluate(model, test_set, batch_size, criterion, ep = 0, qat_mode = False, qat_params = None):
  test_loader = torch.utils.data.DataLoader(dataset = test_set, batch_size = batch_size, shuffle=True, num_workers=1)
  test_iterator = tqdm(test_loader, desc = 'Eval Iteration for epoch:'+str(ep+1), ncols = 900)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  
  model.eval()
  
  if qat_mode:
    weights_fp = clone_model_weights(model)
    w_fp_min_max, w_q_min_max = quantize_model(model, weights_fp, qat_params)
  
  global_step = 0
  total_correct = 0
  total_samples = 0
  total_loss = 0.0
  with torch.no_grad():
    for step, inputs in enumerate(test_iterator):
      global_step +=1
      x, y = inputs[0].to(device), torch.squeeze(inputs[1],1).long().to(device)

      logits = model(x)
      loss = criterion(logits, y)
      correct, samples = get_accuracy(logits, y)
      total_correct +=correct.item()
      total_samples +=samples
      total_loss +=loss
  acc = total_correct / total_samples
  total_loss = total_loss / global_step
  model.train()
  
  return (total_loss, acc)

In [31]:
# Defining the model name and directory
qat_model_name = "qat_model.pt"
model_dir = 'model_artifacts'

In [32]:
qat_params = {'quant_method': 'normal_rounding','bin_method' : 'prior_normal','precision' : 4}

qat_model = DenseNeuralNet(input_size = input_dim, 
                          num_classes = output_classes,
                          layers = [10],
                          dropout_prob=0,
                          batch_norm=False) 

qat_train(qat_model, train_set, val_set, test_set , 
          batch_size = batch_size, 
          learning_rate = learning_rate,
          epochs = epochs, 
          eval_steps = eval_steps, 
          skip_train_set = False,
          qat_mode = True, 
          qat_params = qat_params)

torch.save(qat_model, os.path.join(model_dir, qat_model_name))

print('Running QAT evaluation')
val_loss, val_accuracy = qat_evaluate(qat_model, val_set, batch_size, criterion, ep = 0, qat_mode = True, qat_params = qat_params)
print('Accuracy from quantized model = %f'%val_accuracy)


Step = 714, validation loss =  0.427, validation accuracy = 0.801



Step = 714, training loss =  0.412, training accuracy = 0.812



Step = 1428, validation loss =  0.405, validation accuracy = 0.820



Step = 1428, training loss =  0.391, training accuracy = 0.825



Step = 2142, validation loss =  0.385, validation accuracy = 0.831


...


Step = 6426, training loss =  0.353, training accuracy = 0.857



Step = 7140, validation loss =  0.360, validation accuracy = 0.852



Step = 7140, training loss =  0.348, training accuracy = 0.858




End of Training, test loss =  0.361, test accuracy = 0.849
Running QAT evaluation



Accuracy from quantized model = 0.856467


In [33]:
loss, accuracy = evaluate(model=model, test_set=test_set, batch_size=16, criterion=criterion, ep=0)
print("Accuracy for normal model:",accuracy)
loss, accuracy = qat_evaluate(model=qat_model, test_set=test_set, batch_size=16, criterion=criterion, ep=0)
print("Accuracy for Quantized model:",accuracy)


Accuracy for normal model: 0.8588328075709779



Accuracy for Quantized model: 0.8533123028391167
